# birads_preprocessing


In [ ]:
import os
import numpy as np
import pydicom
from PIL import Image, ImageOps
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Apply windowing to enhance image contrast based on Window Center (WC) and Window Width (WW)
def apply_windowing(img_data, wc, ww):
    try:
        # Calculate minimum and maximum intensity values based on windowing
        min_val, max_val = wc - ww // 2, wc + ww // 2
        # Clip pixel values between min and max and normalize to a range of 0-255
        img_data = np.clip(img_data, min_val, max_val)
        # Return the windowed image data as 8-bit unsigned integer values
        return ((img_data - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    except Exception as e:
        # Print error message if windowing fails
        print(f"Error in apply_windowing: {e}")
        return None

# Convert a DICOM file to a PNG file
def convert_dicom_to_png(input_file, output_file):
    try:
        # Read DICOM file
        ds = pydicom.dcmread(input_file)
        # Extract pixel data from the DICOM file
        img_data = ds.pixel_array
        # Get the Window Center (WC) and Window Width (WW) from DICOM metadata
        wc = ds.get("WindowCenter", img_data.mean())
        ww = ds.get("WindowWidth", img_data.max() - img_data.min())
        # Apply windowing to the image
        img_data = apply_windowing(img_data, wc if isinstance(wc, (int, float)) else wc[0], ww if isinstance(ww, (int, float)) else ww[0])
        # If windowing was successful, save the image as a PNG
        if img_data is not None:
            image = Image.fromarray(img_data)  # Convert to PIL Image object
            image.save(output_file)  # Save as PNG
    except Exception as e:
        # Print error message if DICOM to PNG conversion fails
        print(f"Error converting DICOM to PNG: {e}")

# Check if the image is light (average pixel value > 128)
def is_image_light(image_path):
    try:
        # Open the image file
        image = Image.open(image_path)
        # Convert the image to grayscale
        image_gray = ImageOps.grayscale(image)
        # Convert the grayscale image to a NumPy array
        image_array = np.array(image_gray)
        # Calculate the average pixel value of the grayscale image
        avg_pixel_value = np.mean(image_array)
        # Return True if the average pixel value is greater than 128, indicating a light image
        return avg_pixel_value > 128
    except Exception as e:
        # Print error message if the lightness check fails
        print(f"Error in checking image lightness: {e}")
        return False

# Process an image: convert DICOM to PNG, invert if light, and delete the DICOM file
def process_image(dicom_path, png_path):
    try:
        # Convert DICOM file to PNG
        convert_dicom_to_png(dicom_path, png_path)
        # If the PNG file exists and the image is light, invert the colors
        if os.path.exists(png_path) and is_image_light(png_path):
            image = Image.open(png_path)  # Open the PNG image
            image_inverted = ImageOps.invert(image.convert('RGB'))  # Invert the image
            image_inverted.save(png_path)  # Save the inverted image
        # Remove the original DICOM file after processing
        os.remove(dicom_path)
    except Exception as e:
        # Print error message if image processing fails
        print(f"Error processing image {dicom_path}: {e}")

# Process all DICOM images in a folder
def process_images_in_folder(folder_path):
    # Use ThreadPoolExecutor to process images in parallel with 4 threads
    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust the number of workers as needed
        futures = []  # List to store future tasks
        # Iterate through each subfolder in the specified folder
        for subfolder in tqdm(os.listdir(folder_path)):
            subfolder_path = os.path.join(folder_path, subfolder)  # Full path of the subfolder
            if os.path.isdir(subfolder_path):  # Check if it's a directory
                # Iterate through each file in the subfolder
                for file_name in os.listdir(subfolder_path):
                    # Process only DICOM files (.dcm)
                    if file_name.lower().endswith('.dcm'):
                        dicom_path = os.path.join(subfolder_path, file_name)  # Full path of the DICOM file
                        png_path = os.path.join(subfolder_path, file_name.replace('.dcm', '.png'))  # Path for the PNG file
                        # Submit the process_image task to the thread pool
                        futures.append(executor.submit(process_image, dicom_path, png_path))
        # Wait for all the tasks to complete
        for future in tqdm(futures):
            try:
                future.result()  # Retrieve the result of each task
            except Exception as e:
                # Print error message if any task fails
                print(f"Error in processing a future: {e}")

if __name__ == "__main__":
    # Specify the folder containing the DICOM files
    folder_path = r"C:\Users\zafer\Desktop\competition_dataset\competition_dataset"
    # Process the DICOM files in the folder
    process_images_in_folder(folder_path)
    # Print a message once processing is complete
    print("Image processing complete.")


# Image Cropping

In [ ]:
# Import necessary libraries
import random
import os
import cv2
import matplotlib.pyplot as plt

# Function to read labels from a text file
def etiketleri_okumak(txt_konumu):
    labels = []
    # Open the label file in read mode
    with open(txt_konumu, 'r') as file:
        # Read each line from the file
        for line in file:
            # Convert the line into a list of floating-point numbers
            numbers = list(map(float, line.split()))
            # Append the list of numbers (labels) to the labels list
            labels.append(numbers)
    # Return the list of labels
    return labels

# Function to convert compressed labels to actual coordinates for bounding boxes
def Sikistirilmis_etiketleri_acmak(sikistirilmis_etiketler, image):
    '''The order: x1, x2, y1, y2'''
    etiketler = []
    # Loop through each label (class, x, y, width, height) from compressed labels
    for label in sikistirilmis_etiketler:
        # Unpack label components
        sinif, x_zip, y_zip, genislik_bbox_zip, yukseklik_bbox_zip = label

        # Get image dimensions
        img_height, img_width, rgb = image.shape
        
        # Convert normalized coordinates back to pixel values
        x = x_zip * img_width
        y = y_zip * img_height
        bbox_genislik = genislik_bbox_zip * img_width
        bbox_yukseklik = yukseklik_bbox_zip * img_height

        # Calculate the top-left (x1, y1) and bottom-right (x2, y2) corner coordinates of the bounding box
        x1 = int(x - bbox_genislik / 2)
        x2 = int(x + bbox_genislik / 2)
        y1 = int(y - bbox_yukseklik / 2)
        y2 = int(y + bbox_yukseklik / 2)
        
        # Append the bounding box coordinates to the etiketler list
        etiketler.append([x1, x2, y1, y2])
    # Return bounding box coordinates
    return x1, x2, y1, y2

# Function to make predictions using the model
def Predict_frame(frame, model):
    # Use the model to make predictions on the frame
    predictions = model(frame, save_txt=None)
    labels = []
    
    # Loop through each prediction (bounding box in xywhn format)
    for idx, prediction in enumerate(predictions[0].boxes.xywhn):
        # Get the class of the object detected
        cls = int(predictions[0].boxes.cls[idx].item())
        
        # Format the label as "class x y width height"
        line = f"{cls} {prediction[0].item()} {prediction[1].item()} {prediction[2].item()} {prediction[3].item()}"
        
        # Convert the string to a list of floating-point numbers
        numbers = list(map(float, line.split()))
        
        # Append the numbers to the labels list
        labels.append(numbers)
        
        # Return the labels (since the return statement is inside the loop, it will return after the first prediction)
        return labels

# Import necessary libraries for image processing and multithreading
import os
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to process an image (crop, resize, and save)
def process_image(image_path):
    try:
        # Read the original image
        orginal_image = cv2.imread(image_path)
        # Read the image again for processing (in case further operations modify it)
        image = cv2.imread(image_path)
        
        # Resize the image to 640x640 for prediction
        resized = cv2.resize(image, (640, 640))
        
        # Predict the labels using the model (e.g., object detection)
        labels = Predict_frame(frame=resized, model=MemeModeli)
        
        # Get the actual coordinates for cropping from compressed labels
        x1, x2, y1, y2 = Sikistirilmis_etiketleri_acmak(image=image, sikistirilmis_etiketler=labels)
        
        # Crop the original image based on the bounding box coordinates
        cropped_image = orginal_image[y1:y2, x1:x2]
        
        # Resize the cropped image to 512x512
        resized = cv2.resize(cropped_image, (512, 512))
        
        # Replace the original image with the cropped and resized image
        cv2.imwrite(image_path, resized)

        # Return the path of the processed image
        return image_path
    
    except AttributeError as e:
        # Print error if there is an attribute-related issue
        print(f"AttributeError encountered for {image_path}: {e}")
        return None
    except Exception as e:
        # Print general error if image processing fails
        print(f"Error processing {image_path}: {e}")
        return None

# Function to process all images in the root folder
def process_images_in_root_folder(root_folder):
    # Get a list of subfolders within the root folder
    subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]
    total_subfolders = len(subfolders)
    
    # Process each subfolder one by one
    for i, subfolder in enumerate(subfolders, 1):
        print(f"Processing subfolder {i}/{total_subfolders}: {subfolder}")
        
        # Get the full path of the subfolder
        subfolder_path = os.path.join(root_folder, subfolder)
        
        # Get the full paths of all images within the subfolder
        image_paths = [os.path.join(subfolder_path, image_name) for image_name in os.listdir(subfolder_path)]
        
        # Use multithreading to process images in parallel
        with ThreadPoolExecutor() as executor:
            # Submit each image processing task to the thread pool
            futures = {executor.submit(process_image, image_path): image_path for image_path in image_paths}
            
            # Track the progress of each image processing task
            for future in as_completed(futures):
                # Get the path of the processed image
                processed_image_path = future.result()
                if processed_image_path is not None:
                    # Print message indicating successful processing
                    print(f"Processed {processed_image_path}")

# Set the root folder path where images are stored
root_folder = r'C:\Users\zafer\Desktop\Biradscompetition_dataset\competition_dataset'

# Process all images within the root folder
process_images_in_root_folder(root_folder)


# Prediction

In [ ]:
import os
import pandas as pd
from ultralytics import YOLO

# Define the paths
model_path = r"C:\Users\zafer\Desktop\biradsyoloWeight\best.pt"
images_directory = r"C:\Users\zafer\Desktop\Biradscompetition_dataset\competition_dataset"
output_csv = r"C:\Users\zafer\Desktop\biradSimage_predictions.csv"

# Load your model
model = YOLO(model_path)

# List to hold the results
results_list = []

# Iterate over all images in the directory
for filename in os.listdir(images_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        image_path = os.path.join(images_directory, filename)

        try:
            # Run prediction
            results = model(image_path)

            # Extract the top-1 class name
            top1_index = results[0].probs.top1  # Get the top-1 predicted class index
            class_name = results[0].names[top1_index]  # Convert index to class name

            # Append the results to the list
            results_list.append({
                'filename': filename,
                'kategori': class_name
            })

            print(f"Image: {filename}, Detected class: {class_name}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

# Create a DataFrame from the results list
df = pd.DataFrame(results_list)

# Save the DataFrame to a CSV file
df.to_csv(output_csv, index=False)

print(f"Results saved to {output_csv}")
